In [2]:
from pymongo import MongoClient

In [3]:
db = MongoClient()

In [27]:
for row in db.datasets.postnauka.find().limit(1):
    print(type(row))

<class 'dict'>


In [56]:
row = next(db.datasets.habrahabr.find().limit(1))

In [58]:
row.keys()

dict_keys(['_id', 'modalities', 'title', 'url', 'markdown', 'authors_names'])

In [112]:
db.datasets.postnauka.drop_index("markdown_text")

In [125]:
db.datasets.postnauka.drop_index("title_text")

In [126]:
%%time

db.datasets.postnauka.create_index([("markdown", "text")], default_language='russian')

'markdown_text'

In [127]:
for row in db.datasets.postnauka.find({"authors_names": {"$elemMatch": {"$eq": "Константин Воронцов"}}}):
    print(row["title"])

Математические методы прогнозирования объемов продаж


In [137]:
%%time

for row in db.datasets.postnauka.find({'$text': {'$search': 'анализы данных'}},
                           { "score": { "$meta": "textScore" }}).limit(5).sort([('score', {'$meta': 'textScore'})]):
    print(row["_id"], row["title"], row["score"])

pn_1709 Анализ данных как область знания 2.062361139347119
pn_2537 Анализ данных 2.0473438112134357
pn_1887 FAQ: Анализ данных 2.0436088319431223
pn_681 Главы | Контроль 2.030854885057471
pn_658 Анализ социальных сетей в интернете 2.0223983893293065
CPU times: user 8.07 ms, sys: 4.04 ms, total: 12.1 ms
Wall time: 69.6 ms
